# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv file created in Part I
weather_csv = "../WeatherPy/output_data/city_weather_data.csv"
# Create dataframe
weather_df = pd.read_csv(weather_csv)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hami,42.8000,93.4500,18.81,84.0,0.0,4.74,CN,1.611879e+09
1,luau,-10.7073,22.2247,62.01,99.0,57.0,1.63,AO,1.611879e+09
2,kapaa,22.0752,-159.3190,75.20,69.0,90.0,19.57,US,1.611879e+09
3,albany,42.6001,-73.9662,17.01,58.0,39.0,3.00,US,1.611879e+09
4,arraial do cabo,-22.9661,-42.0278,77.86,81.0,99.0,25.46,BR,1.611879e+09
...,...,...,...,...,...,...,...,...,...
571,sunbury,-37.5833,144.7333,64.00,100.0,75.0,11.50,AU,1.611879e+09
572,bowen,-20.0167,148.2333,82.40,74.0,75.0,16.11,AU,1.611879e+09
573,atambua,-9.1061,124.8925,76.46,89.0,100.0,12.44,ID,1.611879e+09
574,kieta,-6.2167,155.6333,85.39,72.0,100.0,12.95,PG,1.611879e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)
# print(len(humidity_rate))
# # Create a Humidity heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, 
                                 point_radius = 2)

fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_weather_df = weather_df

# Drop any rows that don't contain all three conditions- 
# max temperature lower than 80 degrees but higher than 70
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 80) & (ideal_weather_df["Max Temp"] > 70)]

# wind speed less than 10 mph.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,chicama,-7.8447,-79.1469,71.60,78.0,0.0,8.05,PE,1.611879e+09
192,caravelas,-17.7125,-39.2481,76.46,83.0,0.0,6.24,BR,1.611879e+09
225,sao filipe,14.8961,-24.4956,73.29,66.0,0.0,5.35,CV,1.611879e+09
412,gunjur,13.2019,-16.7339,73.40,53.0,0.0,6.91,GM,1.611879e+09
462,saint-pierre,-21.3393,55.4781,77.00,69.0,0.0,8.05,RE,1.611879e+09
491,alta floresta,-9.8756,-56.0861,73.40,100.0,0.0,4.61,BR,1.611879e+09
492,zabid,14.1951,43.3152,70.75,68.0,0.0,1.28,YE,1.611879e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,chicama,-7.8447,-79.1469,71.60,78.0,0.0,8.05,PE,1.611879e+09
192,caravelas,-17.7125,-39.2481,76.46,83.0,0.0,6.24,BR,1.611879e+09
225,sao filipe,14.8961,-24.4956,73.29,66.0,0.0,5.35,CV,1.611879e+09
412,gunjur,13.2019,-16.7339,73.40,53.0,0.0,6.91,GM,1.611879e+09
462,saint-pierre,-21.3393,55.4781,77.00,69.0,0.0,8.05,RE,1.611879e+09
491,alta floresta,-9.8756,-56.0861,73.40,100.0,0.0,4.61,BR,1.611879e+09
492,zabid,14.1951,43.3152,70.75,68.0,0.0,1.28,YE,1.611879e+09


In [8]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from dataframe
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 151: chicama.
Closest hotel is Hospedaje Punta Cana PERU.
------------
Retrieving Results for Index 192: caravelas.
Closest hotel is Pousada dos Navegantes.
------------
Retrieving Results for Index 225: sao filipe.
Closest hotel is Tortuga B&B.
------------
Retrieving Results for Index 412: gunjur.
Closest hotel is Kuba Lodge.
------------
Retrieving Results for Index 462: saint-pierre.
Closest hotel is Lindsey Hôtel.
------------
Retrieving Results for Index 491: alta floresta.
Closest hotel is Floresta Amazonica Hotel.
------------
Retrieving Results for Index 492: zabid.
Closest hotel is دار الضيافة.
------------


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
